## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
# already set up on Expanse; toggle for colab

# pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] pyfaidx

Set path prefix for Expanse

In [1]:
path_prefix = "/expanse/lustre/projects/nji102/sgriesmer/"

Import SNP datasets

In [2]:
import pandas as pd

abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/SNP_PRVCS_PAT_0_11436-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')
abs_diff_plus_log_odds_scores_norm.head(), abs_diff_plus_log_odds_scores_norm.shape


(   BroadDnd41CtcfUniPk151-ran  BroadDnd41Ezh239875UniPk151-ran  \
 0                   -0.183754                         3.424810   
 1                    0.064347                        -0.568338   
 2                   -0.187900                         0.022428   
 3                   -0.188072                        -0.579203   
 4                   -0.188218                         1.562255   
 
    BroadGm12878CtcfUniPk151-ran  BroadGm12878Ezh239875UniPk151-ran  \
 0                      2.694036                           2.412509   
 1                     -0.206848                          -0.833275   
 2                     -0.236223                          -0.094407   
 3                     -0.207058                          -0.549511   
 4                     -0.240059                          -0.354813   
 
    BroadH1hescChd1a301218aUniPk151-ran  BroadH1hescCtcfUniPk151-ran  \
 0                            -0.411801                    -0.204063   
 1                      

Choose test set for run

In [3]:
import xgboost

/cm/shared/apps/spack/0.17.3/cpu/b/opt/spack/linux-rocky8-zen/gcc-8.5.0/anaconda3-2021.05-q4munrgvh7qp4o7r3nzcdkbuph4z7375/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot

In [5]:
X=abs_diff_plus_log_odds_scores_norm[0:11436]

In [6]:
X

,BroadDnd41CtcfUniPk151-ran,BroadDnd41Ezh239875UniPk151-ran,BroadGm12878CtcfUniPk151-ran,BroadGm12878Ezh239875UniPk151-ran,BroadH1hescChd1a301218aUniPk151-ran,BroadH1hescCtcfUniPk151-ran,BroadH1hescEzh239875UniPk151-ran,BroadH1hescJarid1aab26049UniPk151-ran,BroadH1hescRbbp5a300109aUniPk151-ran,BroadHelas3CtcfUniPk151-ran,...,UwMcf7CtcfUniPk151-ran.1,UwNb4CtcfUniPk151-ran.1,UwNhdfneoCtcfUniPk151-ran.1,UwNhekCtcfUniPk151-ran.1,UwNhlfCtcfUniPk151-ran.1,UwRptecCtcfUniPk151-ran.1,UwSaecCtcfUniPk151-ran.1,UwSknshraCtcfUniPk151-ran.1,UwWerirb1CtcfUniPk151-ran.1,UwWi38CtcfUniPk151-ran.1
0,-0.183754,3.424810,2.694036,2.412509,-0.411801,-0.204063,0.261441,-0.444878,2.368303,-0.197179,...,3.500897,2.239102,0.042683,-0.080850,0.187946,3.808217,0.558647,-0.270571,1.675248,0.313500
1,0.064347,-0.568338,-0.206848,-0.833275,-0.448292,0.222933,-0.298382,-0.538764,-0.389391,-0.176765,...,2.066476,0.157434,-0.409339,-0.116471,-0.302672,-0.204659,-0.364293,-0.256381,-0.025908,-0.045311
2,-0.187900,0.022428,-0.236223,-0.094407,-0.447457,-0.208221,-0.079009,-0.438841,-0.381472,-0.184461,...,0.104195,0.000767,-0.097367,0.338750,-0.327467,-0.371372,-0.180069,0.036038,-0.223213,0.556211
3,-0.188072,-0.579203,-0.207058,-0.549511,-0.448281,-0.209791,-0.295054,-0.533341,-0.387631,-0.214296,...,0.538721,-0.273758,-0.256114,0.820249,-0.345126,-0.168301,-0.257199,0.351504,-0.254524,0.284484
4,-0.188218,1.562255,-0.240059,-0.354813,0.046917,-0.209931,0.623560,-0.508516,0.290801,-0.214543,...,-0.293038,-0.258978,0.137079,-0.293672,-0.129454,0.213571,0.347450,-0.274081,-0.274353,-0.299367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11431,1.436435,2.034629,-0.110084,-0.118079,0.770749,0.515963,-0.298379,0.941044,-0.345420,-0.179089,...,0.037913,0.305890,-0.381551,0.591475,0.195190,-0.324202,0.088541,2.264096,-0.279252,-0.317476
11432,1.256410,1.516462,6.572957,-0.719056,-0.371734,4.279061,-0.298381,-0.293700,1.352434,-0.213418,...,3.183404,8.288656,0.271697,0.782883,0.296712,-0.342937,-0.098434,0.176402,-0.134688,-0.343947
11433,-0.187023,-0.507361,-0.142247,-0.389903,-0.037496,-0.172706,-0.298380,-0.534680,0.162450,-0.200429,...,-0.369160,-0.044679,-0.390402,-0.249210,-0.197178,-0.323037,-0.501625,-0.311546,-0.243426,-0.392906
11434,-0.182650,-0.332720,-0.180204,0.637317,-0.387532,1.854642,-0.298372,-0.186441,0.040595,-0.143796,...,0.189565,0.659720,-0.184623,0.929336,-0.028026,0.036826,-0.104785,-0.265703,0.182184,-0.188974


In [7]:
col_lab = list(range(0,1380))
X = X.set_axis(col_lab, axis="columns")

In [8]:
X

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.183754,3.424810,2.694036,2.412509,-0.411801,-0.204063,0.261441,-0.444878,2.368303,-0.197179,...,3.500897,2.239102,0.042683,-0.080850,0.187946,3.808217,0.558647,-0.270571,1.675248,0.313500
1,0.064347,-0.568338,-0.206848,-0.833275,-0.448292,0.222933,-0.298382,-0.538764,-0.389391,-0.176765,...,2.066476,0.157434,-0.409339,-0.116471,-0.302672,-0.204659,-0.364293,-0.256381,-0.025908,-0.045311
2,-0.187900,0.022428,-0.236223,-0.094407,-0.447457,-0.208221,-0.079009,-0.438841,-0.381472,-0.184461,...,0.104195,0.000767,-0.097367,0.338750,-0.327467,-0.371372,-0.180069,0.036038,-0.223213,0.556211
3,-0.188072,-0.579203,-0.207058,-0.549511,-0.448281,-0.209791,-0.295054,-0.533341,-0.387631,-0.214296,...,0.538721,-0.273758,-0.256114,0.820249,-0.345126,-0.168301,-0.257199,0.351504,-0.254524,0.284484
4,-0.188218,1.562255,-0.240059,-0.354813,0.046917,-0.209931,0.623560,-0.508516,0.290801,-0.214543,...,-0.293038,-0.258978,0.137079,-0.293672,-0.129454,0.213571,0.347450,-0.274081,-0.274353,-0.299367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11431,1.436435,2.034629,-0.110084,-0.118079,0.770749,0.515963,-0.298379,0.941044,-0.345420,-0.179089,...,0.037913,0.305890,-0.381551,0.591475,0.195190,-0.324202,0.088541,2.264096,-0.279252,-0.317476
11432,1.256410,1.516462,6.572957,-0.719056,-0.371734,4.279061,-0.298381,-0.293700,1.352434,-0.213418,...,3.183404,8.288656,0.271697,0.782883,0.296712,-0.342937,-0.098434,0.176402,-0.134688,-0.343947
11433,-0.187023,-0.507361,-0.142247,-0.389903,-0.037496,-0.172706,-0.298380,-0.534680,0.162450,-0.200429,...,-0.369160,-0.044679,-0.390402,-0.249210,-0.197178,-0.323037,-0.501625,-0.311546,-0.243426,-0.392906
11434,-0.182650,-0.332720,-0.180204,0.637317,-0.387532,1.854642,-0.298372,-0.186441,0.040595,-0.143796,...,0.189565,0.659720,-0.184623,0.929336,-0.028026,0.036826,-0.104785,-0.265703,0.182184,-0.188974


In [9]:
import pandas as pd

snp_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/PAT_dataset_XY-named.csv", sep=',')


In [10]:
y = snp_prvcs['label'][0:11436]

In [11]:
y

0        0
1        0
2        0
3        0
4        0
        ..
11431    1
11432    1
11433    1
11434    1
11435    1
Name: label, Length: 11436, dtype: int64

In [12]:
y[860:870]

860    0
861    0
862    0
863    0
864    0
865    0
866    0
867    0
868    0
869    0
Name: label, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
test_size = 0.3
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

Try logistic regression

In [14]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='lbfgs', max_iter=1000)
lr_train_model = lr_model.fit(X_train,y_train)
pred_lr = lr_model.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score

print("Accuracy_model_lr: %.4f" % (accuracy_score(y_test, pred_lr))) 
print("F1_model_lr: %.4f" % (f1_score(y_test, pred_lr)))
print("MCC_model_lr: %.4f" % (matthews_corrcoef(y_test, pred_lr)))
print("ROC_AUC_model_lr: %.4f" % (roc_auc_score(y_test, pred_lr))) # don't know about call: y_score vs. pred

Accuracy_model_lr: 0.8852
F1_model_lr: 0.3498
MCC_model_lr: 0.2916
ROC_AUC_model_lr: 0.6654


In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred_lr)
roc_auc = roc_auc_score(y_test, pred_lr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [ ]:
from sklearn.metrics import confusion_matrix

#confusion_matrix(y_val, pred1)
tn, fp, fn, tp = confusion_matrix(y_test, pred_lr).ravel()
print(tp, fp, fn, tn)

In [ ]:
# Save logistic regression model

from joblib import dump, load

best_estimator = lr_train_model
dump(best_estimator, path_prefix + "DNABERT_2/Output_Models/" + "model_xpg_boost_PRVCS_PAT_lr.joblib")

In [16]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score

model1 = XGBClassifier() 
model2 = XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5) 
train_model1 = model1.fit(X_train, y_train) 
train_model2 = model2.fit(X_train, y_train) 
pred1 = train_model1.predict(X_test) 
pred2 = train_model2.predict(X_test) 
print("Accuracy_model1: %.4f" % (accuracy_score(y_test, pred1))) 
print("Accuracy_model2: %.4f" % (accuracy_score(y_test, pred2))) 
print("F1_model1: %.4f" % (f1_score(y_test, pred1)))
print("F1_model2: %.4f" % (f1_score(y_test, pred2)))
print("MCC_model1: %.4f" % (matthews_corrcoef(y_test, pred1)))
print("MCC_model2: %.4f" % (matthews_corrcoef(y_test, pred2)))
print("ROC_AUC_model1: %.4f" % (roc_auc_score(y_test, pred1))) # don't know about call: y_score vs. pred
print("ROC_AUC_model2: %.4f" % (roc_auc_score(y_test, pred2))) # don't know about call: y_score vs. pred


Accuracy_model1: 0.9423
Accuracy_model2: 0.9435
F1_model1: 0.4107
F1_model2: 0.4192
MCC_model1: 0.4759
MCC_model2: 0.4910
ROC_AUC_model1: 0.6312
ROC_AUC_model2: 0.6336


In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, pred1).ravel()
print(tp, fp, fn, tn)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred1)
roc_auc = roc_auc_score(y_test, pred1)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred2).ravel()
print(tp, fp, fn, tn)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred2)
roc_auc = roc_auc_score(y_test, pred2)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [ ]:
model3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=7)

train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)
print("Accuracy_model3: %.4f" % (accuracy_score(y_test, pred3)))
print("F1_model3: %.4f" % (f1_score(y_test, pred3)))
print("MCC_model3: %.4f" % (matthews_corrcoef(y_test, pred3)))
print("ROC_AUC_model3: %.4f" % (roc_auc_score(y_test, pred3))) # don't know about call: y_score vs. pred

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred3).ravel()
print(tp, fp, fn, tn)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred3)
roc_auc = roc_auc_score(y_test, pred3)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [24]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[10],
 'min_child_weight':[1],
 'learning_rate':[0.1],
 'n_estimators':[1000],
 'objective':['binary:logistic']
 #'objective':['binary:logistic','binary:hinge','binary:logitraw']
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(gamma=0, 
                                               subsample=0.8, 
                                               colsample_bytree=0.8, 
                                               nthread=4, 
                                               scale_pos_weight=1, 
                                               seed=7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=5,
                       refit=True)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
print("Accuracy_model4: %.4f" % (accuracy_score(y_test, pred4)))  
print("F1_model4: %.4f" % (f1_score(y_test, pred4)))  
print("MCC_model4: %.4f" % (matthews_corrcoef(y_test, pred4)))  
print("ROC_AUC_model4: %.4f" % (roc_auc_score(y_test, pred4)))  

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Accuracy_model4: 0.9437
F1_model4: 0.4239
MCC_model4: 0.4947
ROC_AUC_model4: 0.6355


In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred4).ravel()
print(tp, fp, fn, tn)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred4)
roc_auc = roc_auc_score(y_test, pred4)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [ ]:
gsearch.cv_results_['params'][gsearch.best_index_]

In [ ]:
model_deepsea1 = XGBClassifier(
 reg_alpha = 20,
 reg_lambda = 2000,
 eta = 0.1,
 n_estimators=10,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea1 = model_deepsea1.fit(X_train, y_train)
pred_deepsea1 = train_model_deepsea1.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea1)))
print("F1_model_deepsea1: %.4f" % (f1_score(y_test, pred_deepsea1)))
print("MCC_model_deepsea1: %.4f" % (matthews_corrcoef(y_test, pred_deepsea1)))
print("ROC_AUC_model_deepsea1: %.4f" % (roc_auc_score(y_test, pred_deepsea1))) # don't know about call: y_score vs. pred

In [ ]:
model_deepsea2 = XGBClassifier(
 reg_alpha = 0,
 reg_lambda = 10,
 eta = 0.1,
 n_estimators=500,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea2 = model_deepsea2.fit(X_train, y_train)
pred_deepsea2 = train_model_deepsea2.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea2)))
print("F1_model_deepsea2: %.4f" % (f1_score(y_test, pred_deepsea2)))
print("MCC_model_deepsea2: %.4f" % (matthews_corrcoef(y_test, pred_deepsea2)))
print("ROC_AUC_model_deepsea2: %.4f" % (roc_auc_score(y_test, pred_deepsea2))) # don't know about call: y_score vs. pred

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred_deepsea2).ravel()
print(tp, fp, fn, tn)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred_deepsea2)
roc_auc = roc_auc_score(y_test, pred_deepsea2)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [18]:
model_regbase = XGBClassifier(
    colsample_bylevel = 0.3,
    colsample_bytree = 1.0,
    gamma = 3.6,
    learning_rate = 0.1,
    max_depth = 8,
    min_child_weight = 3,
    n_estimators = 100,
    reg_alpha = 0.001,
    reg_lambda = 2,
    scale_pos_weight = 12.1599539701,
    subsample = 0.7)

train_model_regbase = model_regbase.fit(X_train, y_train)
pred_regbase = train_model_regbase.predict(X_test)
print("Accuracy_model_regbase: %.4f" % (accuracy_score(y_test, pred_regbase)))
print("F1_model_regbase: %.4f" % (f1_score(y_test, pred_regbase)))
print("MCC_model_regbase: %.4f" % (matthews_corrcoef(y_test, pred_regbase)))
print("ROC_AUC_model_regbase: %.4f" % (roc_auc_score(y_test, pred_regbase)))

Accuracy_model_regbase: 0.9414
F1_model_regbase: 0.4806
MCC_model_regbase: 0.4876
ROC_AUC_model_regbase: 0.6730


In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred_regbase).ravel()
print(tp, fp, fn, tn)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred_regbase)
roc_auc = roc_auc_score(y_test, pred_regbase)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [19]:
import xgboost as xgb

data_matrix = xgb.DMatrix(data=X, label=y)

params = {
    "colsample_bylevel": 0.3,
    "colsample_bytree": 1.0,
    "gamma": 3.6,
    "learning_rate": 0.1,
    "max_depth": 8,
    "min_child_weight": 3,
    "alpha": 0.001,
    "lambda": 2,
    "scale_pos_weight": 12.1599539701,
    "subsample": 0.7
}

xgb_cv = xgb.cv(dtrain=data_matrix, params=params, nfold=10,
                    num_boost_round=100, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=7)


In [20]:
xgb_cv

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.850845,0.010008,0.669715,0.028620
1,0.919368,0.004921,0.710552,0.023180
2,0.946874,0.004257,0.719757,0.030174
3,0.960756,0.003113,0.731105,0.026727
4,0.969932,0.003279,0.743583,0.023903
5,0.975509,0.002432,0.745466,0.025259
6,0.979075,0.001572,0.747188,0.022923
7,0.981869,0.001708,0.749809,0.022561
8,0.984163,0.001740,0.749290,0.023436
9,0.985835,0.002152,0.752434,0.023256


In [21]:
model_regbase.set_params(n_estimators=xgb_cv.shape[0])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.3, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=3.6, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=27, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [22]:
train_model_regbase_cv = model_regbase.fit(X_train,y_train)
pred_regbase_cv = train_model_regbase.predict(X_test)
print("Accuracy_model_regbase: %.4f" % (accuracy_score(y_test, pred_regbase_cv)))
print("F1_model_regbase: %.4f" % (f1_score(y_test, pred_regbase_cv)))
print("MCC_model_regbase: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv)))
print("ROC_AUC_model_regbase: %.4f" % (roc_auc_score(y_test, pred_regbase_cv)))

Accuracy_model_regbase: 0.9131
F1_model_regbase: 0.4087
MCC_model_regbase: 0.3622
ROC_AUC_model_regbase: 0.6752


In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred_regbase_cv).ravel()
print(tp, fp, fn, tn)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, pred_regbase_cv)
roc_auc = roc_auc_score(y_test, pred_regbase_cv)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot(name="ROC Curve")

In [ ]:
pred_regbase.sum()

In [ ]:
param_test = {
 'max_depth': range(2,10,2),
 'min_child_weight': range(2,10,2)
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(colsample_bylevel = 0.3,
                                                colsample_bytree = 1.0,
                                                gamma = 3.6,
                                                learning_rate = 0.1,
                                                max_depth = 8,
                                                min_child_weight = 3,
                                                n_estimators = 27,
                                                objective = 'binary:logistic',
                                                reg_alpha = 0.001,
                                                reg_lambda = 2,
                                                scale_pos_weight = 12.1599539701,
                                                subsample = 0.7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=10,
                       refit=True)

train_model_regbase_cv3 = gsearch.fit(X_train, y_train)
pred_regbase_cv3 = train_model_regbase_cv3.predict(X_test)
print("Accuracy_model_cv3: %.4f" % (accuracy_score(y_test, pred_regbase_cv3)))  
print("F1_model_cv3: %.4f" % (f1_score(y_test, pred_regbase_cv3)))  
print("MCC_model_cv3: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv3)))  
print("ROC_AUC_model_cv3: %.4f" % (roc_auc_score(y_test, pred_regbase_cv3)))  

In [ ]:
gsearch.cv_results_['params'][gsearch.best_index_]

In [ ]:
param_test = {
 'max_depth': [7,8,9],
 'min_child_weight': [7,8,9]
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(colsample_bylevel = 0.3,
                                                colsample_bytree = 1.0,
                                                gamma = 3.6,
                                                learning_rate = 0.1,
                                                max_depth = 8,
                                                min_child_weight = 3,
                                                n_estimators = 27,
                                                objective = 'binary:logistic',
                                                reg_alpha = 0.001,
                                                reg_lambda = 2,
                                                scale_pos_weight = 12.1599539701,
                                                subsample = 0.7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=10,
                       refit=True)

train_model_regbase_cv4 = gsearch.fit(X_train, y_train)
pred_regbase_cv4 = train_model_regbase_cv4.predict(X_test)
print("Accuracy_model_cv4: %.4f" % (accuracy_score(y_test, pred_regbase_cv4)))  
print("F1_model_cv4: %.4f" % (f1_score(y_test, pred_regbase_cv4)))  
print("MCC_model_cv4: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv4)))  
print("ROC_AUC_model_cv4: %.4f" % (roc_auc_score(y_test, pred_regbase_cv4)))  

In [ ]:
gsearch.cv_results_['params'][gsearch.best_index_]

In [ ]:
pd.DataFrame(gsearch.cv_results_)

In [ ]:
param_test = {
 'gamma': range(2,6),
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(colsample_bylevel = 0.3,
                                                colsample_bytree = 1.0,
                                                gamma = 3.6,
                                                learning_rate = 0.1,
                                                max_depth = 9,
                                                min_child_weight = 9,
                                                n_estimators = 27,
                                                objective = 'binary:logistic',
                                                reg_alpha = 0.001,
                                                reg_lambda = 2,
                                                scale_pos_weight = 12.1599539701,
                                                subsample = 0.7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=10,
                       refit=True)

train_model_regbase_cv5 = gsearch.fit(X_train, y_train)
pred_regbase_cv5 = train_model_regbase_cv5.predict(X_test)
print("Accuracy_model_cv5: %.4f" % (accuracy_score(y_test, pred_regbase_cv5)))  
print("F1_model_cv5: %.4f" % (f1_score(y_test, pred_regbase_cv5)))  
print("MCC_model_cv5: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv5)))  
print("ROC_AUC_model_cv5: %.4f" % (roc_auc_score(y_test, pred_regbase_cv5))) 

In [ ]:
gsearch.cv_results_['params'][gsearch.best_index_]

In [ ]:
pd.DataFrame(gsearch.cv_results_)

In [25]:
param_test = {
  'subsample': [i/10.0 for i in range(6,10)],
  'colsample_bytree': [i/10.0 for i in range(6,10)]
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(colsample_bylevel = 0.3,
                                                colsample_bytree = 1.0,
                                                gamma = 3.6,
                                                learning_rate = 0.1,
                                                max_depth = 9,
                                                min_child_weight = 9,
                                                n_estimators = 27,
                                                objective = 'binary:logistic',
                                                reg_alpha = 0.001,
                                                reg_lambda = 2,
                                                scale_pos_weight = 12.1599539701,
                                                subsample = 0.7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=10,
                       refit=True)

train_model_regbase_cv6 = gsearch.fit(X_train, y_train)
pred_regbase_cv6 = train_model_regbase_cv6.predict(X_test)
print("Accuracy_model_cv6: %.4f" % (accuracy_score(y_test, pred_regbase_cv6)))  
print("F1_model_cv6: %.4f" % (f1_score(y_test, pred_regbase_cv6)))  
print("MCC_model_cv6: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv6)))  
print("ROC_AUC_model_cv6: %.4f" % (roc_auc_score(y_test, pred_regbase_cv6))) 

Fitting 10 folds for each of 16 candidates, totalling 160 fits
Accuracy_model_cv6: 0.9231
F1_model_cv6: 0.4454
MCC_model_cv6: 0.4066
ROC_AUC_model_cv6: 0.6859


In [26]:
gsearch.cv_results_['params'][gsearch.best_index_]

{'colsample_bytree': 0.7, 'subsample': 0.9}

In [ ]:
pd.DataFrame(gsearch.cv_results_)

In [ ]:
param_test = {
  'reg_alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1],
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(colsample_bylevel = 0.3,
                                                colsample_bytree = 0.7,
                                                gamma = 3.6,
                                                learning_rate = 0.1,
                                                max_depth = 9,
                                                min_child_weight = 9,
                                                n_estimators = 27,
                                                objective = 'binary:logistic',
                                                reg_alpha = 0.001,
                                                reg_lambda = 2,
                                                scale_pos_weight = 12.1599539701,
                                                subsample = 0.9), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=10,
                       refit=True)

train_model_regbase_cv7 = gsearch.fit(X_train, y_train)
pred_regbase_cv7 = train_model_regbase_cv7.predict(X_test)
print("Accuracy_model_cv7: %.4f" % (accuracy_score(y_test, pred_regbase_cv7)))  
print("F1_model_cv7: %.4f" % (f1_score(y_test, pred_regbase_cv7)))  
print("MCC_model_cv7: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv7)))  
print("ROC_AUC_model_cv7: %.4f" % (roc_auc_score(y_test, pred_regbase_cv7))) 

In [ ]:
gsearch.cv_results_['params'][gsearch.best_index_]

In [ ]:
pd.DataFrame(gsearch.cv_results_)

In [ ]:
param_test = {
  'reg_alpha':[5e-3, 3e-3, 1e-3, 8e-2, 6e-2],
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(colsample_bylevel = 0.3,
                                                colsample_bytree = 0.7,
                                                gamma = 3.6,
                                                learning_rate = 0.1,
                                                max_depth = 9,
                                                min_child_weight = 9,
                                                n_estimators = 27,
                                                objective = 'binary:logistic',
                                                reg_alpha = 0.001,
                                                reg_lambda = 2,
                                                scale_pos_weight = 12.1599539701,
                                                subsample = 0.9), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=10,
                       refit=True)

train_model_regbase_cv8 = gsearch.fit(X_train, y_train)
pred_regbase_cv8 = train_model_regbase_cv8.predict(X_test)
print("Accuracy_model_cv8: %.4f" % (accuracy_score(y_test, pred_regbase_cv8)))  
print("F1_model_cv8: %.4f" % (f1_score(y_test, pred_regbase_cv8)))  
print("MCC_model_cv8: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv8)))  
print("ROC_AUC_model_cv8: %.4f" % (roc_auc_score(y_test, pred_regbase_cv8))) 

In [ ]:
gsearch.cv_results_['params'][gsearch.best_index_]

In [ ]:
pd.DataFrame(gsearch.cv_results_)

In [ ]:
param_test = {
  'learning_rate':[0.01, 0.05, 0.1, 0.15, 0.2],
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(colsample_bylevel = 0.3,
                                                colsample_bytree = 0.7,
                                                gamma = 3.6,
                                                learning_rate = 0.1,
                                                max_depth = 9,
                                                min_child_weight = 9,
                                                n_estimators = 27,
                                                objective = 'binary:logistic',
                                                reg_alpha = 0.001,
                                                reg_lambda = 2,
                                                scale_pos_weight = 12.1599539701,
                                                subsample = 0.9), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=10,
                       refit=True)

train_model_regbase_cv9 = gsearch.fit(X_train, y_train)
pred_regbase_cv9 = train_model_regbase_cv9.predict(X_test)
print("Accuracy_model_cv9: %.4f" % (accuracy_score(y_test, pred_regbase_cv9)))  
print("F1_model_cv9: %.4f" % (f1_score(y_test, pred_regbase_cv9)))  
print("MCC_model_cv9: %.4f" % (matthews_corrcoef(y_test, pred_regbase_cv9)))  
print("ROC_AUC_model_cv9: %.4f" % (roc_auc_score(y_test, pred_regbase_cv9))) 

In [ ]:
gsearch.cv_results_['params'][gsearch.best_index_]

In [ ]:
pd.DataFrame(gsearch.cv_results_)

In [ ]:
# Save default model

from joblib import dump, load

best_estimator = train_model1
dump(best_estimator, path_prefix + "DNABERT_2/Output_Models/" + "model_xpg_boost_PRVCS_PAT_default.joblib")

In [ ]:
# Save best model

In [ ]:
from joblib import dump, load

best_estimator = train_model_regbase_cv6
dump(best_estimator, path_prefix + "DNABERT_2/Output_Models/" + "model_xpg_boost_PRVCS_PAT_all.joblib")

In [ ]:
from joblib import dump, load

test_estimator = load('/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_PAT_all.joblib')

In [ ]:
test_pred = test_estimator.predict(X_test)

In [ ]:
test_pred.sum()

In [ ]:
# Load Rate_Patho dataset

In [ ]:
RARPAT_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/RARPAT_0_102-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [ ]:
RARPAT_abs_diff_plus_log_odds_scores_norm.shape

In [ ]:
# Set model names to numbers

In [ ]:
RARPAT_abs_diff_plus_log_odds_scores_norm = RARPAT_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns")

In [ ]:
RARPAT_abs_diff_plus_log_odds_scores_norm

In [ ]:
RARPAT_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/test_dataset/Rare_Patho_SNV_dataset_XY-named.txt", sep='\t')

In [ ]:
y = RARPAT_prvcs['label']

In [ ]:
y

In [ ]:
RARPAT_pred = test_estimator.predict(RARPAT_abs_diff_plus_log_odds_scores_norm)

In [ ]:
print("Accuracy_model_CAN: %.4f" % (accuracy_score(y, RARPAT_pred)))  
print("F1_model_CAN: %.4f" % (f1_score(y, RARPAT_pred)))  
print("MCC_model_CAN: %.4f" % (matthews_corrcoef(y, RARPAT_pred)))  
print("ROC_AUC_model_CAN: %.4f" % (roc_auc_score(y , RARPAT_pred)))

Load ASD dataset

In [ ]:
ASD_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/ASD_0_107-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [ ]:
ASD_abs_diff_plus_log_odds_scores_norm = ASD_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns")

In [ ]:
ASD_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/test_dataset/ASD_denovo_SNV_dataset_XY-named.txt", sep='\t')

In [ ]:
y = ASD_prvcs['label']

In [ ]:
ASD_pred = test_estimator.predict(ASD_abs_diff_plus_log_odds_scores_norm)

In [ ]:
print("Accuracy_model_CAN: %.4f" % (accuracy_score(y, ASD_pred)))  
print("F1_model_CAN: %.4f" % (f1_score(y, ASD_pred)))  
print("MCC_model_CAN: %.4f" % (matthews_corrcoef(y, ASD_pred)))  
print("ROC_AUC_model_CAN: %.4f" % (roc_auc_score(y , ASD_pred)))

Load GTEx dataset

In [ ]:
GTEX_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/GTEX_0_796-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [ ]:
GTEX_abs_diff_plus_log_odds_scores_norm = GTEX_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns")

In [ ]:
GTEX_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/test_dataset/GTEx_eQTL_dataset_XY-named.txt", sep='\t')

In [ ]:
y = GTEX_prvcs['label']

In [ ]:
GTEX_pred = test_estimator.predict(GTEX_abs_diff_plus_log_odds_scores_norm)

In [ ]:
print("Accuracy_model_CAN: %.4f" % (accuracy_score(y, GTEX_pred)))  
print("F1_model_CAN: %.4f" % (f1_score(y, GTEX_pred)))  
print("MCC_model_CAN: %.4f" % (matthews_corrcoef(y, GTEX_pred)))  
print("ROC_AUC_model_CAN: %.4f" % (roc_auc_score(y , GTEX_pred)))

Load Somatic eQTL dataset

In [ ]:
SOMEQTL_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/SOMEQTL_0_7513-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [ ]:
SOMEQTL_abs_diff_plus_log_odds_scores_norm = SOMEQTL_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns")

In [ ]:
SOMEQTL_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/test_dataset/Somatic_eQTL_dataset_XY-named.txt", sep='\t')

In [ ]:
y = SOMEQTL_prvcs['label']

In [ ]:
SOMEQTL_pred = test_estimator.predict(SOMEQTL_abs_diff_plus_log_odds_scores_norm)

In [ ]:
print("Accuracy_model_CAN: %.4f" % (accuracy_score(y, SOMEQTL_pred)))  
print("F1_model_CAN: %.4f" % (f1_score(y, SOMEQTL_pred)))  
print("MCC_model_CAN: %.4f" % (matthews_corrcoef(y, SOMEQTL_pred)))  
print("ROC_AUC_model_CAN: %.4f" % (roc_auc_score(y , SOMEQTL_pred)))